In [1]:
#requires python 3.10 for compatability with python-obd library

In [2]:
import obd
#pyserial is a dependency of obd but does not need to be imported
from obd import OBDStatus
import logging
import serial.tools.list_ports
import time
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QVBoxLayout, QWidget, QCheckBox, QPushButton, QHBoxLayout, QStackedWidget
from PyQt5.QtCore import Qt
import os
import json

# List USB Ports

In [3]:
#if it needs to be set manually the top usb 3.0 port 
#Bus 001 Device 007: ID 0403:6015 Future Technology Devices International, Ltd Bridge(I2C/SPI/UART/FIFO)


# Create Interface

In [2]:
'''
class PIDSelectionPage(QMainWindow):
    def __init__(self):
        try:
            super().__init__()
            self.setWindowTitle("PID Selection")
            self.showFullScreen()  # Launch in full-screen mode

            # Main layout
            self.layout = QVBoxLayout()

            # Top bar layout with Quit button
            self.top_bar_layout = QHBoxLayout()

            # Quit button
            self.quit_button = QPushButton("Quit")
            self.quit_button.clicked.connect(self.close_application)
            self.top_bar_layout.addWidget(self.quit_button)

            # Save Active Selections button
            self.save_button = QPushButton("Save Active Selections")
            self.save_button.clicked.connect(self.save_active_selections)
            self.top_bar_layout.addWidget(self.save_button)

            # Add stretch to push buttons to the left
            self.top_bar_layout.addStretch()
            self.layout.addLayout(self.top_bar_layout)

            # OBD Connect and Check Supported PIDs buttons
            self.connect_button = QPushButton("Connect to OBD")
            self.connect_button.clicked.connect(self.connect_to_obd)
            self.layout.addWidget(self.connect_button)

            self.check_pids_button = QPushButton("Check Vehicle Supported PIDs")
            self.check_pids_button.clicked.connect(self.check_supported_pids)
            self.layout.addWidget(self.check_pids_button)

            # Table widget for displaying PIDs
            self.table_widget = QTableWidget()
            self.layout.addWidget(self.table_widget)
            self.central_widget = QWidget()
            self.central_widget.setLayout(self.layout)
            self.setCentralWidget(self.central_widget)

            self.connection = None
            self.initUI()

        except Exception as e:
            logging.error("Error during initialization: %s", e)
            raise  # Re-raise the exception after logging

    def initUI(self):
        # Populate the table with PIDs from module 1
        all_pids = {name: getattr(obd.commands, name) for name in dir(obd.commands) if not name.startswith("__")}
        module_one_pids = {name: cmd for name, cmd in all_pids.items() if isinstance(cmd, obd.OBDCommand) and cmd.mode == 1}

        self.table_widget.setColumnCount(6)  # PID, Name, Description, Response Value, Active, Supported
        self.table_widget.setHorizontalHeaderLabels(["PID", "Name", "Description", "Response Value", "Active", "Supported"])

        self.table_widget.setRowCount(len(module_one_pids))
        for row, (pid_name, pid_cmd) in enumerate(sorted(module_one_pids.items(), key=lambda x: x[1].pid)):
            self.table_widget.setItem(row, 0, QTableWidgetItem(str(pid_cmd.pid)))  # PID
            self.table_widget.setItem(row, 1, QTableWidgetItem(pid_cmd.name))  # Name
            self.table_widget.setItem(row, 2, QTableWidgetItem(pid_cmd.desc))  # Description
            self.table_widget.setItem(row, 3, QTableWidgetItem(''))  # Response Value

            # Add a checkbox in the 'Active' column
            checkbox = QCheckBox()
            checkbox.setCheckState(Qt.Unchecked)
            self.table_widget.setCellWidget(row, 4, checkbox)

            # Add a label in the 'Supported' column
            supported_label = QTableWidgetItem('No')
            supported_label.setFlags(Qt.ItemIsEnabled)  # Make it non-editable
            self.table_widget.setItem(row, 5, supported_label)

        # Ensure all cells have a QTableWidgetItem
        for row in range(self.table_widget.rowCount()):
            for column in range(self.table_widget.columnCount()):
                if not self.table_widget.item(row, column):
                    self.table_widget.setItem(row, column, QTableWidgetItem(''))

        self.load_cached_pids()
        self.load_active_selections()

        self.table_widget.resizeColumnsToContents()

    def load_cached_pids(self):
        cache_file = "supported_pids.json"
        if os.path.exists(cache_file):
            # Load supported PIDs from cache
            with open(cache_file, "r") as file:
                supported_pids = json.load(file)
            self.update_table_with_supported_pids(supported_pids)
        else:
            # Enable all PIDs initially
            for row in range(self.table_widget.rowCount()):
                checkbox = self.table_widget.cellWidget(row, 4)
                if checkbox:
                    checkbox.setCheckState(Qt.Checked)

    def close_application(self):
        self.save_active_selections()  # Save the current state of the "Active" checkboxes
        if self.connection:
            self.connection.close()  # Close the OBD connection
        self.close()  # Close the application

    def save_active_selections(self):
        active_selections = {}
        for row in range(self.table_widget.rowCount()):
            pid_name = self.table_widget.item(row, 1).text()
            checkbox = self.table_widget.cellWidget(row, 4)
            active_selections[pid_name] = checkbox.isChecked()

        with open("active_selections.json", "w") as file:
            json.dump(active_selections, file)

    def connect_to_obd(self):
        self.connection = obd.OBD()  # Try to connect to OBD-II adapter

    def check_supported_pids(self):
        cache_file = "supported_pids.json"
        if self.connection and self.connection.is_connected():
            supported_pids = self.get_supported_pids()
            # Cache the supported PIDs
            with open(cache_file, "w") as file:
                json.dump(supported_pids, file)
            self.update_table_with_supported_pids(supported_pids)
        elif os.path.exists(cache_file):
            # Load supported PIDs from cache if available
            with open(cache_file, "r") as file:
                supported_pids = json.load(file)
            self.update_table_with_supported_pids(supported_pids)

    def get_supported_pids(self):
        supported_pids = []
        for cmd in obd.commands:
            if self.connection.supports(cmd):
                supported_pids.append(cmd.name)
        return supported_pids

    def update_table_with_supported_pids(self, supported_pids):
        for row in range(self.table_widget.rowCount()):
            pid_name = self.table_widget.item(row, 1).text()
            supported_label = self.table_widget.item(row, 5)
            active_checkbox = self.table_widget.cellWidget(row, 4)

            if pid_name in supported_pids:
                supported_label.setText('Yes')
                active_checkbox.setEnabled(True)  # Enable the checkbox if PID is supported
            else:
                supported_label.setText('No')
                active_checkbox.setCheckState(Qt.Unchecked)  # Uncheck the checkbox
                active_checkbox.setEnabled(False)  # Disable the checkbox if PID is not supported

    def load_active_selections(self):
        if os.path.exists("active_selections.json"):
            with open("active_selections.json", "r") as file:
                active_selections = json.load(file)

            for row in range(self.table_widget.rowCount()):
                pid_name = self.table_widget.item(row, 1).text()
                checkbox = self.table_widget.cellWidget(row, 4)
                checkbox.setChecked(active_selections.get(pid_name, False))

def main():
    app = QApplication(sys.argv)
    ex = PIDSelectionPage()
    ex.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

SystemExit: 0

c:\Users\jonathan.cascioli\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# V0

In [3]:

class PIDSelectionPage(QMainWindow):

    def __init__(self):
        ''' 
        Initialize the PIDSelectionPage class and set up the main GUI components.

        This method sets up the main window, its layout, and the primary widgets 
        including the top bar with control buttons and the navigation layout. 
        It also initializes a QStackedWidget for managing different pages within the interface.
        '''

        super().__init__()
        
        self.setWindowTitle("PID Selection")
        self.showFullScreen()  # Launch in full-screen mode

        # Main layout
        self.layout = QVBoxLayout()

        # Top bar layout with Quit button
        self.top_bar_layout = QHBoxLayout()

        # Quit button
        self.quit_button = QPushButton("Quit")
        self.quit_button.clicked.connect(self.close_application)
        self.top_bar_layout.addWidget(self.quit_button)

        # Save Active Selections button
        self.save_button = QPushButton("Save Active Selections")
        self.save_button.clicked.connect(self.save_active_selections)
        self.top_bar_layout.addWidget(self.save_button)

        # Add stretch to push buttons to the left
        self.top_bar_layout.addStretch()
        self.layout.addLayout(self.top_bar_layout)

        # Navigation buttons layout
        self.nav_layout = QHBoxLayout()
        self.add_nav_buttons()
        self.layout.addLayout(self.nav_layout)

        # Stacked widget for different pages
        self.stacked_widget = QStackedWidget()
        self.layout.addWidget(self.stacked_widget)
        self.init_pages()

        self.central_widget = QWidget()
        self.central_widget.setLayout(self.layout)
        self.setCentralWidget(self.central_widget)

        self.connection = None

    def add_nav_buttons(self):

        """
        Add navigation buttons to the GUI for switching between different pages. 
        This method creates buttons for the Cluster Panel, PID Selection, DTC Log, and Error/Warning Log. 
        Each button is connected to a lambda function that changes the current index of the QStackedWidget, enabling navigation between different pages.
        """

        
        # Buttons for navigation
        self.cluster_button = QPushButton("Cluster Panel")
        self.cluster_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(0))
        self.nav_layout.addWidget(self.cluster_button)

        self.pid_button = QPushButton("PID Selection")
        self.pid_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(1))
        self.nav_layout.addWidget(self.pid_button)

        self.dtc_button = QPushButton("DTC Log")
        self.dtc_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(2))
        self.nav_layout.addWidget(self.dtc_button)

        self.log_button = QPushButton("Error/Warning Log")
        self.log_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(3))
        self.nav_layout.addWidget(self.log_button)

    def init_pages(self):
        """
        Initialize different pages to be displayed in the QStackedWidget.
        This method sets up various pages including Cluster, PID Selection, DTC Log, 
        and Error/Warning Log. It uses a dedicated method for initializing each page. 
        All these pages are then added to the QStackedWidget.
        """

        # Initialize the Cluster Panel page
        self.cluster_page = self.init_cluster_panel()  # Replace placeholder with actual initialization

        # Initialize the PID Selection page
        self.pid_page = self.init_pid_page()  # The current PID selection page

        # Initialize the DTC Log page
        self.dtc_page = QWidget()  # Placeholder for the DTC log page

        # Initialize the Error/Warning Log page
        self.log_page = QWidget()  # Placeholder for the Error/Warning log page

        # Add pages to the stacked widget
        self.stacked_widget.addWidget(self.cluster_page)
        self.stacked_widget.addWidget(self.pid_page)
        self.stacked_widget.addWidget(self.dtc_page)
        self.stacked_widget.addWidget(self.log_page)

    def init_pid_page(self):
        """
        Set up the PID selection page.
        This method creates the layout for the PID selection page, including buttons 
        for connecting to OBD and checking supported PIDs, and a QTableWidget for 
        displaying PID information. It returns the QWidget representing the PID 
        selection page.
        """

        # Set up the PID selection page (what you have been working on)
        pid_page = QWidget()
        pid_layout = QVBoxLayout(pid_page)

        # OBD Connect and Check Supported PIDs buttons
        connect_button = QPushButton("Connect to OBD")
        connect_button.clicked.connect(self.connect_to_obd)
        pid_layout.addWidget(connect_button)

        check_pids_button = QPushButton("Check Vehicle Supported PIDs")
        check_pids_button.clicked.connect(self.check_supported_pids)
        pid_layout.addWidget(check_pids_button)

        # Table widget for displaying PIDs
        self.table_widget = QTableWidget()
        pid_layout.addWidget(self.table_widget)
        self.init_pid_table()

        return pid_page


    def init_pid_table(self):
        """
        Initialize the PID table with columns, headers, and data.

        This method sets up the columns and headers for the PID table and populates 
        it with data. It also includes checkboxes for PID selection and handles 
        loading of supported and active PID selections from cached files.
        """

        # Set up the columns and headers for the PID table
        self.table_widget.setColumnCount(6)  # PID, Name, Description, Response Value, Active, Supported
        self.table_widget.setHorizontalHeaderLabels(["PID", "Name", "Description", "Response Value", "Active", "Supported"])

        # Example: populate the table with PIDs (this part will depend on your specific implementation)
        all_pids = {name: getattr(obd.commands, name) for name in dir(obd.commands) if not name.startswith("__")}
        module_one_pids = {name: cmd for name, cmd in all_pids.items() if isinstance(cmd, obd.OBDCommand) and cmd.mode == 1}
        
        self.table_widget.setRowCount(len(module_one_pids))
        for row, (pid_name, pid_cmd) in enumerate(sorted(module_one_pids.items(), key=lambda x: x[1].pid)):
            self.table_widget.setItem(row, 0, QTableWidgetItem(str(pid_cmd.pid)))  # PID
            self.table_widget.setItem(row, 1, QTableWidgetItem(pid_cmd.name))  # Name
            self.table_widget.setItem(row, 2, QTableWidgetItem(pid_cmd.desc))  # Description
            self.table_widget.setItem(row, 3, QTableWidgetItem(''))  # Response Value

            # Add a checkbox in the 'Active' column
            checkbox = QCheckBox()
            checkbox.setCheckState(Qt.Unchecked)
            self.table_widget.setCellWidget(row, 4, checkbox)

            # Add a label in the 'Supported' column
            supported_label = QTableWidgetItem('No')
            supported_label.setFlags(Qt.ItemIsEnabled)  # Make it non-editable
            self.table_widget.setItem(row, 5, supported_label)
        
        self.table_widget.resizeColumnsToContents()
        
        #load supported pids
        self.load_cached_pids()
        # Load active selections after table is fully populated
        self.load_active_selections()

    def init_cluster_panel(self):
        """
        Initialize the Cluster Panel page.
        
        This method sets up the layout and widgets for the Cluster Panel, 
        where real-time data from the vehicle will be displayed.
        """
        # Initialize your widgets and layout here
        self.cluster_panel = QWidget()
        self.cluster_layout = QVBoxLayout(self.cluster_panel)
        # Add widgets like labels, tables, graphs, etc., to display OBD data


    def save_active_selections(self):
        """
        Save the currently active PID selections to a JSON file.

        This method iterates through the rows of the PID table, collecting the state 
        of each checkbox, and then saves this data to 'active_selections.json'.
        """

        active_selections = {}
        for row in range(self.table_widget.rowCount()):
            pid_name = self.table_widget.item(row, 1).text()
            checkbox = self.table_widget.cellWidget(row, 4)
            active_selections[pid_name] = checkbox.isChecked()

        with open("active_selections.json", "w") as file:
            json.dump(active_selections, file)


    def connect_to_obd(self, async_connection=False, manual_protocol_set="AUTO"):
        """
        Handle the connection to the OBD-II adapter, either synchronously or asynchronously.

        Args:
            async_connection (bool): Determines whether to establish an asynchronous connection.
        """

        def on_connect(connection):
            if connection.is_connected():
                logging.info("Successfully connected to OBDLink EX.")
                # Introduce a 10 second delay before starting async commands
                time.sleep(2)
                self.start_cluster_monitoring()
            else:
                logging.error("Failed to connect to OBDLink EX. Please check the connection.")

        def on_disconnect():
            logging.info("Disconnected from OBDLink EX.")
            self.stop_cluster_monitoring()

        try:
            if async_connection:
                # Establish an asynchronous connection for Cluster Panel
                self.connection = obd.Async(connection = obd.Async(port=None,  # manually set USB port if not working with None
                       fast=False,
                       baudrate=115200,
                       protocol=manual_protocol_set,
                       check_voltage=True))
                
                self.connection.connect(on_connect_callback=on_connect, on_disconnect_callback=on_disconnect)  # Set up connection and disconnection callbacks
            else:
                # Establish a synchronous connection for PID Selection
                self.connection = obd.OBD()  # Current synchronous connection setup
        except Exception as e:
            print(f"Connection error: {e}")


    def check_supported_pids(self):
        """
        Check and update the table with supported PIDs.

        This method determines if a connection is established and retrieves supported 
        PIDs. It caches these PIDs in a file and updates the PID table accordingly. 
        If no connection is available, it attempts to load supported PIDs from the 
        cache file.
        """

        cache_file = "supported_pids.json"
        if self.connection and self.connection.is_connected():
            supported_pids = self.get_supported_pids()
            # Cache the supported PIDs
            with open(cache_file, "w") as file:
                json.dump(supported_pids, file)
            self.update_table_with_supported_pids(supported_pids)
        elif os.path.exists(cache_file):
            # Load supported PIDs from cache if available
            with open(cache_file, "r") as file:
                supported_pids = json.load(file)
            self.update_table_with_supported_pids(supported_pids)

    def get_supported_pids(self):

        ''' 
        Retrieve a list of supported PIDs from the OBD connection.

        This method iterates through all possible OBD commands and checks if each 
        command is supported by the current OBD connection. It compiles a list of 
        supported PID names and returns this list.

        Returns:
            supported_pids (list): A list of supported PID names.
        '''

        supported_pids = []
        for cmd in obd.commands:
            if self.connection.supports(cmd):
                supported_pids.append(cmd.name)
        return supported_pids

    def update_table_with_supported_pids(self, supported_pids):
        ''' 
        Update the PID table to reflect which PIDs are supported.

        This method goes through each row in the PID table and updates the 
        'Supported' column based on whether the PID is in the list of supported PIDs. 
        It also enables or disables the corresponding checkbox in the 'Active' column 
        depending on the PID's support status.

        Args:
            supported_pids (list): A list of supported PID names.
        '''

        for row in range(self.table_widget.rowCount()):
            pid_name = self.table_widget.item(row, 1).text()
            supported_label = self.table_widget.item(row, 5)
            active_checkbox = self.table_widget.cellWidget(row, 4)

            if pid_name in supported_pids:
                supported_label.setText('Yes')
                active_checkbox.setEnabled(True)
            else:
                supported_label.setText('No')
                active_checkbox.setCheckState(Qt.Unchecked)
                active_checkbox.setEnabled(False)

    def load_active_selections(self):
        ''' 
        Load and apply active PID selections from a JSON file.

        This method checks if the 'active_selections.json' file exists. If it does, 
        it loads the active selections and updates the checkboxes in the PID table 
        accordingly. The method ensures that only supported PIDs are checked.

        '''

        if os.path.exists("active_selections.json"):
            with open("active_selections.json", "r") as file:
                active_selections = json.load(file)

            for row in range(self.table_widget.rowCount()):
                pid_name = self.table_widget.item(row, 1).text()
                checkbox = self.table_widget.cellWidget(row, 4)
                supported_label = self.table_widget.item(row, 5).text()

                if supported_label == 'Yes':
                    checkbox.setChecked(active_selections.get(pid_name, False))
                else:
                    checkbox.setChecked(False)
                    checkbox.setEnabled(False)

    
    def load_cached_pids(self):
        ''' 
        Load supported PIDs from a cached file and update the PID table.

        This method checks for the existence of a 'supported_pids.json' cache file. 
        If the file exists, it loads the supported PIDs from the file and updates 
        the table. If the file does not exist, it assumes all PIDs are initially 
        supported and checks all checkboxes in the table.

        '''

        cache_file = "supported_pids.json"
        if os.path.exists(cache_file):
            # Load supported PIDs from cache
            with open(cache_file, "r") as file:
                supported_pids = json.load(file)
            self.update_table_with_supported_pids(supported_pids)
        else:
            # Enable all PIDs initially
            for row in range(self.table_widget.rowCount()):
                checkbox = self.table_widget.cellWidget(row, 4)
                if checkbox:
                    checkbox.setCheckState(Qt.Checked)

    ##############################_________CLUSTER PANEL___________############################################
    ############################################################################################################                

    def switch_to_cluster_panel(self):
        """
        Switch to the Cluster Panel page and manage OBD connections.
        """
        # Close existing synchronous connection if open
        if self.connection and not isinstance(self.connection, obd.Async):
            self.connection.close()

        # Establish an asynchronous connection for Cluster Panel
        self.connect_to_obd(async_connection=True)

        # Switch to the Cluster Panel page
        self.stacked_widget.setCurrentIndex(0)  # Index of your Cluster Panel

    def get_active_pid_selections_from_cache(self):
        """
        Retrieve the active PID selections from the 'active_selections.json' cache file.

        Returns:
            active_pids (list): A list of active PID names.
        """
        active_pids = []
        cache_file = "active_selections.json"
        if os.path.exists(cache_file):
            with open(cache_file, "r") as file:
                active_selections = json.load(file)
            # Filter only active PIDs
            active_pids = [pid for pid, active in active_selections.items() if active]

        return active_pids
    
    def start_cluster_monitoring(self):
        """
        Start monitoring the active PIDs in the Cluster Panel.
        """
        active_pids = self.get_active_pid_selections_from_cache()

        for pid_name in active_pids:
            cmd = getattr(obd.commands, pid_name, None)
            if cmd:
                self.connection.watch(cmd, callback=self.data_callback)
        self.connection.start()  # Start the asynchronous data monitoring

    def stop_cluster_monitoring(self):
        """
        Stop monitoring the PIDs in the Cluster Panel.
        """
        if self.connection.is_connected():
            self.connection.stop()  # Stop the asynchronous data monitoring
            self.connection.unwatch_all()  # Clear all watched commands



    def data_callback(self, response, cmd_name):
        """
        Handle incoming data from the OBD connection.

        Args:
            response: The response from the OBD command.
            cmd_name (str): The name of the OBD command.
        """
        if response.value is not None:
            logging.info(f"{cmd_name}: {response.value}")
            # Update GUI elements here based on the cmd_name and response.value
            # For example, update a label or a graph
        else:
            logging.warning(f"{cmd_name}: No data received")


    

    def close_application(self):
        """
        Handle application closure by saving selections and closing connections.

        This method ensures that active PID selections are saved before closing the 
        application. It also handles the closure of any existing OBD-II connections 
        and then closes the application window.
        """
  
        self.save_active_selections()
        if self.connection:
            self.stop_cluster_monitoring()
        self.close()



def main():
    app = QApplication(sys.argv)
    ex = PIDSelectionPage()
    ex.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


SystemExit: 0

c:\Users\jonathan.cascioli\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Establish Asynch Connection:

Async is a subclass of OBD, and therefore inherits all of the standard methods. However, Async adds a few in order to control a threaded update loop. This loop will keep the values of your commands up to date with the vehicle. This way, when the user querys the car, the latest response is returned immediately.

documentation:
https://python-obd.readthedocs.io/en/latest/Async%20Connections/



In [ ]:
#set to auto if vehicle protocol unknown
manual_protocol_set = "J1850PWM" #1997 ford f360 specific

# Setting up logging for better debugging
logging.basicConfig(level=logging.INFO)

def on_connect(connection):
    if connection.is_connected():
        logging.info("Successfully connected to OBDLink EX.")
        # Introduce a 10 second delay before starting async commands
        time.sleep(10)
        start_monitoring()
    else:
        logging.error("Failed to connect to OBDLink EX. Please check the connection.")

def on_disconnect():
    logging.info("Disconnected from OBDLink EX.")

def start_monitoring():
    # Function to handle callback for each command
    def data_callback(response, cmd_name):
        if response.value is not None:
            logging.info(f"{cmd_name}: {response.value}")
        else:
            logging.warning(f"{cmd_name}: No data received")

    # Setting up monitoring for various OBD commands with delays
    commands = {
        "OIL_TEMP": obd.commands.OIL_TEMP,
        "FUEL_LEVEL": obd.commands.FUEL_LEVEL,
        "RUN_TIME": obd.commands.RUN_TIME,
        "SPEED": obd.commands.SPEED,
        "MAF": obd.commands.MAF,
        "FUEL_PRESSURE": obd.commands.FUEL_PRESSURE,
        "COOLANT_TEMP": obd.commands.COOLANT_TEMP,
        "ENGINE_LOAD": obd.commands.ENGINE_LOAD,
        "CONTROL_MODULE_VOLTAGE": obd.commands.CONTROL_MODULE_VOLTAGE # battery
          }

    for cmd_name, cmd in commands.items():
        connection.watch(cmd, callback=lambda r, cmd_name=cmd_name: data_callback(r, cmd_name))
        time.sleep(0.5)  # Introduce a 500ms delay between each command setup

# Configure the OBD connection
manual_protocol_set = "AUTO"  # Replace with specific protocol if needed
connection = obd.Async(port=None,  # manually set USB port if not working with None
                       fast=False,
                       baudrate=115200,
                       protocol=manual_protocol_set,
                       check_voltage=True)

# Setting up connection and disconnection handlers
connection.connect(on_connect_callback=on_connect, on_disconnect_callback=on_disconnect)

# Keep the script running to maintain connection and handle callbacks
try:
    while True:
        pass
except KeyboardInterrupt:
    # Disconnect before closing the script
    connection.close()
    logging.info("Program terminated.")
